## Transformer Encoder 的每个 Block 结构详解

1. **多头自注意力机制（Multi-Head Self-Attention）**
   - 输入序列首先通过线性变换，分别生成 Query（查询）、Key（键）、Value（值）三个矩阵。
   - 每个位置的 Query 与所有位置的 Key 计算相关性分数（注意力分数），并对 Value 加权求和，得到新的表示。
   - 多头机制将输入拆分为多个子空间，分别计算注意力，最后拼接结果，增强模型表达能力。

2. **残差连接与 LayerNorm**
   - 自注意力输出与原输入做残差连接（相加），再进行 Layer Normalization（层归一化），提升训练稳定性，加速收敛。

3. **前馈全连接网络（Feed Forward Network, FFN）**
   - 每个位置的输出分别通过两层线性变换和激活函数（如 ReLU 或 GELU），结构通常为：`Linear -> Activation -> Linear`。
   - 进一步提升每个位置的特征表达能力。

4. **再次残差连接与 LayerNorm**
   - FFN 输出与自注意力后的结果做残差连接，再进行 Layer Normalization，最终输出每个 block 的结果。

---

**整体结构流程图：**

输入  
↓  
位置编码(Positional Encoding)  
↓   
多头自注意力（Multi-Head Self-Attention）  
↓  
残差连接 + LayerNorm  
↓  
前馈网络（Feed Forward Network）  
↓  
残差连接 + LayerNorm  
↓  
输出

## 位置编码Positional_Encoding

你这段 `PositionalEncoding` 是 Transformer **Encoder** 里常用的**位置编码（Positional Encoding）**实现，核心作用是：
给纯粹的词向量（word embedding）加入**位置信息**，让模型能区分“我爱你”和“你爱我”这种单词顺序不同的句子。

下面我按结构拆开讲：

---

### 1. **为什么需要位置编码？**

Transformer 的 **Self-Attention** 机制本身是**顺序无关**的，它只看 token 间的相似度，不知道谁在前谁在后。
所以必须给每个 token 加一个**位置向量**（position vector），这样模型才知道 token 在句子里的位置。

---

### 2. **代码执行流程**

#### (1) 创建空矩阵

```python
pe = torch.zeros(max_len, embed_size)
```

* `max_len`：句子最大长度（5000个位置）。
* `embed_size`：词向量维度。
* 每行代表一个位置（pos），每列代表该位置对应的一个维度。

---

#### (2) 生成位置索引

```python
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
```

* 得到 `[0, 1, 2, ..., max_len-1]` 的列向量，表示位置 ID。
* `unsqueeze(1)` 把它变成 `(max_len, 1)` 方便广播计算。

---

#### (3) 生成频率缩放因子

```python
div_term = torch.exp(
    torch.arange(0, embed_size, 2).float() * (-math.log(10000.0) / embed_size)
)
```

* `torch.arange(0, embed_size, 2)`：取偶数下标（0, 2, 4...）。
* 缩放因子公式：

  $$
  \frac{1}{10000^{2i / d_{\text{model}}}}
  $$

  这是论文 *Attention is All You Need* 提出的，让不同维度的波形频率不同，便于模型捕捉短距离和长距离的依赖。

---

#### (4) 计算 sin/cos 位置编码

```python
pe[:, 0::2] = torch.sin(position * div_term)  # 偶数列
pe[:, 1::2] = torch.cos(position * div_term)  # 奇数列
```

* **偶数维**：`sin(pos / 10000^(2i/d))`
* **奇数维**：`cos(pos / 10000^(2i/d))`
* 为什么用 sin/cos？

  * 周期性 → 可以推广到比训练时更长的序列。
  * 不同维度有不同周期 → 模型可线性组合得到任何相对位置。

---

#### (5) 形状调整

```python
pe = pe.unsqueeze(0)  # (1, max_len, embed_size)
```

* 多加一个 batch 维度，方便和 `(batch_size, seq_len, embed_size)` 的输入对齐。

---

#### (6) 注册 buffer

```python
self.register_buffer('pe', pe)
```

* `pe` 不是参数（不会被梯度更新），但会随模型保存/加载。
* 避免被误认为是可训练权重。

---

#### (7) 前向传播

```python
x = x + self.pe[:, :x.size(1), :]
```

* 取出和当前句子长度 `seq_len` 匹配的那一段位置编码。
* 直接 **逐元素相加** 到词向量 embedding 上，得到带位置信息的输入。

---

### 3. **整体过程总结**

**输入**：词向量矩阵 `(batch_size, seq_len, embed_size)`
**输出**：加入位置编码的词向量 `(batch_size, seq_len, embed_size)`

> 原理上，Encoder 在送进多头注意力之前，先用：
>
> $$
> x_{\text{pos}} = x_{\text{embed}} + PE
> $$
>
> 这样 Attention 就能利用相对位置信息。





In [1]:
import torch
import torch.nn as nn
import math

# 位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, max_len=5000):
        super().__init__()
        # 创建一个全零矩阵，形状为 (max_len, embed_size)，用于存储每个位置的编码
        pe = torch.zeros(max_len, embed_size)
        # 生成位置索引，形状为 (max_len, 1)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # 计算每个维度的缩放因子，公式来自论文
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * (-math.log(10000.0) / embed_size))
        # 偶数维度用 sin，奇数维度用 cos，生成不同频率的波形
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # 增加 batch 维度，变成 (1, max_len, embed_size)
        pe = pe.unsqueeze(0)
        # 注册为 buffer，不参与训练，但会随模型保存
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (batch_size, seq_len, embed_size)
        # 取出和输入序列长度匹配的那一段位置编码，并加到输入上
        x = x + self.pe[:, :x.size(1), :]
        return x

## 多头自注意力机制

(N, seq_len, embed_size)  
&nbsp;&nbsp;&nbsp;&nbsp;↓ Q/K 投影到 qk_dim，V 保持 embed_size  
(N, seq_len, qk_dim) / (N, seq_len, embed_size)  
&nbsp;&nbsp;&nbsp;&nbsp;↓ 多头拆分  
(N, heads, seq_len, head_dim_qk) / (N, heads, seq_len, head_dim_v)  
&nbsp;&nbsp;&nbsp;&nbsp;↓ 注意力计算  
(N, heads, seq_len, head_dim_v)  
&nbsp;&nbsp;&nbsp;&nbsp;↓ 多头合并  
(N, seq_len, embed_size)  
&nbsp;&nbsp;&nbsp;&nbsp;↓ 输出线性层  
(N, seq_len, embed_size)

In [ ]:
# 多头自注意力机制
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads, qk_dim):
        """
        embed_size: 输入 token 向量维度（d_model）
        heads: 注意力头数
        qk_dim: Q/K 投影后的维度（会均分到每个头）
        """
        super().__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim_qk = qk_dim // heads           # 每个头的 Q/K 维度
        self.head_dim_v = embed_size // heads        # 每个头的 V 维度（保持和输入一致）

        assert qk_dim % heads == 0, "qk_dim 必须能被 heads 整除"
        assert embed_size % heads == 0, "embed_size 必须能被 heads 整除"

        # Q/K 的线性映射到 qk_dim
        self.query = nn.Linear(embed_size, qk_dim)
        self.key = nn.Linear(embed_size, qk_dim)
        # V 保持 embed_size 维
        self.value = nn.Linear(embed_size, embed_size)

        # 输出线性层
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, query, key=None, value=None, mask=None):
        """
        query: (N, seq_len_q, embed_size)
        key:   (N, seq_len_k, embed_size)，默认 None，则用 query
        value: (N, seq_len_v, embed_size)，默认 None，则用 query
        mask:  注意力 mask，可选
        """
        if key is None:
            key = query
        if value is None:
            value = query

        N = query.shape[0]
        query_len = query.shape[1]
        key_len = key.shape[1]
        value_len = value.shape[1]

        # 1️⃣ Q/K/V 投影
        Q = self.query(query)    # (N, query_len, qk_dim)
        K = self.key(key)        # (N, key_len, qk_dim)
        V = self.value(value)    # (N, value_len, embed_size)

        # 2️⃣ 拆成多头
        Q = Q.view(N, query_len, self.heads, self.head_dim_qk).transpose(1, 2)  # (N, heads, query_len, head_dim_qk)
        K = K.view(N, key_len, self.heads, self.head_dim_qk).transpose(1, 2)    # (N, heads, key_len, head_dim_qk)
        V = V.view(N, value_len, self.heads, self.head_dim_v).transpose(1, 2)   # (N, heads, value_len, head_dim_v)

        # 3️⃣ Scaled Dot-Product Attention
        energy = torch.matmul(Q, K.transpose(-1, -2)) / math.sqrt(self.head_dim_qk)
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        attention = torch.softmax(energy, dim=-1)  # (N, heads, query_len, key_len)

        # 4️⃣ 注意力加权 V
        out = torch.matmul(attention, V)  # (N, heads, query_len, head_dim_v)

        # 5️⃣ 多头拼接
        out = out.transpose(1, 2).contiguous().view(N, query_len, self.embed_size)  # (N, query_len, embed_size)

        # 6️⃣ 输出线性层
        out = self.fc_out(out)  # (N, query_len, embed_size)

        return out

## 前馈神经网络

In [3]:
# 前馈神经网络
class FeedForward(nn.Module):
    def __init__(self, embed_size, hidden_dim, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(embed_size, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, embed_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

## 每个Endoder Block

In [4]:
# Encoder Block
class EncoderBlock(nn.Module):
    def __init__(self, embed_size, heads, qk_dim, ffn_hidden, dropout=0.1):
        super().__init__()
        # 使用通用 SelfAttention
        self.attention = SelfAttention(embed_size, heads, qk_dim)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.ffn = FeedForward(embed_size, ffn_hidden, dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # 1️⃣ 自注意力 + 残差 + LayerNorm
        attn_out = self.attention(query=x, mask=mask)  # Q/K/V 都是 x
        x = self.norm1(x + self.dropout(attn_out))

        # 2️⃣ 前馈网络 + 残差 + LayerNorm
        ffn_out = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_out))

        return x

## Transformer Encoder

In [5]:
# Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_layers, heads, qk_dim,
                 ff_hidden_size, dropout=0.1, max_length=100):
        super().__init__()
        self.embed_size = embed_size
        
        # 1️⃣ 词嵌入
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        
        # 2️⃣ 位置编码
        self.position_encoding = PositionalEncoding(embed_size, max_length)
        
        # 3️⃣ 多层 EncoderBlock
        self.layers = nn.ModuleList([
            EncoderBlock(embed_size, heads, qk_dim, ff_hidden_size, dropout)
            for _ in range(num_layers)
        ])
        
        # 4️⃣ 输入 dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # x: (N, seq_len)
        out = self.word_embedding(x)           # (N, seq_len, embed_size)
        out = self.position_encoding(out)      # 加入位置信息
        out = self.dropout(out)

        # 堆叠多层 EncoderBlock
        for layer in self.layers:
            out = layer(out, mask)             # (N, seq_len, embed_size)

        return out



`vocab_size` 和 `max_length` 看起来都是整数参数，但它们代表的意义完全不同：

---

### 🔹 `vocab_size`

* **定义**：词表大小，即模型能识别的 **不同 token（词/子词/字符）数量**。
* **作用**：作为 `nn.Embedding(vocab_size, embed_size)` 的输入维度。

  * 每个 token ID 是 `[0, vocab_size-1]` 的整数索引
  * Embedding 层会把 ID 映射为一个 `embed_size` 维的向量

👉 举例：

* 如果 `vocab_size = 30,000`，说明模型词表里有 30k 个不同的 token。
* 输入 token ID `42`，Embedding 会查表得到 `word_embedding[42]` 这个向量。

---

### 🔹 `max_length`

* **定义**：模型支持的 **序列最大长度**，通常用于 **位置编码 (Positional Encoding)**。
* **作用**：告诉模型“我最多处理多长的输入序列”。

  * 在 Transformer 里，位置编码是一个 `max_length × embed_size` 的矩阵。
  * 输入序列长度不能超过 `max_length`，否则就没有对应的位置编码。

👉 举例：

* 如果 `max_length = 100`，表示输入/输出序列最长不能超过 100 个 token。
* 当输入句子长 80 时，只用前 80 个位置编码；当输入句子长 120 时，就会报错（超出最大长度）。

---

### ⚖️ 区别总结

| 参数               | 控制什么                  | 决定的限制                |
| ---------------- | --------------------- | -------------------- |
| **`vocab_size`** | 词表的广度（能识别多少个不同 token） | 不能输入超出词表范围的 token id |
| **`max_length`** | 序列的深度（一次能处理多少个 token） | 不能输入超过该长度的序列         |

---

#### 类比理解 🎯

* `vocab_size` 像 **字典里有多少个词**
* `max_length` 像 **一句话最多能有多少个词**




好的，我们来仔细拆解这一行代码：

```python
self.word_embedding = nn.Embedding(vocab_size, embed_size)
```

---

### 🔹 1. `nn.Embedding` 的作用

* 它的本质是一个 **查表层 (lookup table)**。
* 输入：整数索引 (token id)，范围是 `[0, vocab_size-1]`。
* 输出：对应的 **稠密向量** (embedding)，维度为 `embed_size`。

也就是说：

* 模型不会直接处理离散的 token id，而是先把它转成连续的向量。
* 这个向量能捕捉到 token 的语义信息（在训练中学习到）。

---

### 🔹 2. 输入和输出形状

假设定义：

```python
word_embedding = nn.Embedding(vocab_size=10000, embed_size=256)
```

#### 输入 (token id 序列)

* 输入通常是一个 `LongTensor`，形状：

  ```
  (N, seq_len)
  ```

  * `N`: batch size
  * `seq_len`: 序列长度

举例：

```python
x = torch.tensor([[2, 45, 78], [5, 9, 100]])  # (N=2, seq_len=3)
```

表示：

* 第一个样本序列 = \[2, 45, 78]
* 第二个样本序列 = \[5, 9, 100]

---

#### 输出 (embedding 向量序列)

* 输出的形状：

  ```
  (N, seq_len, embed_size)
  ```

  * 每个 token id 被映射成一个 `embed_size` 维的向量

举例：

```python
out = word_embedding(x)  # (2, 3, 256)
```

输出：

* batch 里有 2 个句子
* 每个句子长度 = 3
* 每个 token 变成 256 维向量

所以最终 `out` 是：

```
[
  [ [e_2], [e_45], [e_78]   ],   # 第一个句子的 embedding 序列
  [ [e_5], [e_9],  [e_100] ]    # 第二个句子的 embedding 序列
]
```

其中 `[e_x]` 表示 token x 的 embedding 向量，维度是 `256`。

---

### 🔹 3. 与 Transformer 的关系

* 这是 **Encoder/Decoder 的第一步**。
* 输入 token id → word embedding → 加上 position encoding → 送进后续的 self-attention。

---

⚡ 总结一句：
`nn.Embedding(vocab_size, embed_size)` 就是把 **离散的单词 id (N, seq\_len)** 转成 **稠密的语义向量序列 (N, seq\_len, embed\_size)**，为后续 Transformer 提供连续空间的输入。



| 参数               | 含义                                                    |
| ---------------- | ----------------------------------------------------- |
| `vocab_size`     | 词表大小，用于 nn.Embedding，把 token id 映射成向量                 |
| `embed_size`     | embedding 维度，也就是 d\_model，所有 Encoder 层输入输出维度一致        |
| `num_layers`     | Encoder 堆叠的层数，也就是有多少个 EncoderBlock                    |
| `heads`          | 多头注意力头数，注意力机制会把 Q/K/V 拆成多头                            |
| `qk_dim`         | Q/K 投影后的维度（可以小于 embed\_size），每个头的维度 = qk\_dim / heads |
| `ff_hidden_size` | 前馈网络隐藏层维度，一般比 embed\_size 大，比如 4\~8 倍                 |
| `dropout`        | Dropout 概率，用于防止过拟合                                    |
| `max_length`     | 位置编码最大支持序列长度                                          |


✅ 特点：

- x 是 token id 序列，通过 embedding + position encoding 得到输入向量。
- 每个 EncoderBlock 包含：
  - 多头自注意力（Q/K 可降维，V 保持 embed_size）
  - 前馈网络
  - 残差 + LayerNorm
- 可以堆叠多层，输出维度固定为 (batch, seq_len, embed_size)，便于传入 Decoder 或做下游任务。

## Decoder Block

输入 x  
&nbsp;&nbsp;&nbsp;&nbsp;│  
Masked Self-Attention (Q/K/V = x, mask = tgt_mask)  
&nbsp;&nbsp;&nbsp;&nbsp;│  
残差 + LayerNorm  
&nbsp;&nbsp;&nbsp;&nbsp;│  
Cross-Attention (Q = x, K/V = enc_out, mask = src_mask)  
&nbsp;&nbsp;&nbsp;&nbsp;│  
残差 + LayerNorm  
&nbsp;&nbsp;&nbsp;&nbsp;│  
Feed-Forward Network  
&nbsp;&nbsp;&nbsp;&nbsp;│  
残差 + LayerNorm  
&nbsp;&nbsp;&nbsp;&nbsp;│  
输出 x (N, tgt_seq_len, embed_size)

好的，我们来仔细分析 Transformer Decoder 中的 **`tgt_mask`** 和 **`src_mask`** 的区别和作用。

---

### 1️⃣ `tgt_mask`（Target Mask / Decoder 自注意力掩码）

**用途**：

* 用于 **Masked Self-Attention**，屏蔽 Decoder 当前预测位置能看到的未来 token。
* 目的是 **保证自回归（auto-regressive）生成** 的正确性：模型只能看到当前位置及之前的 token。

**形式**：

* 通常是一个 **下三角矩阵**（seq\_len × seq\_len），也可以广播成 `(N, 1, seq_len, seq_len)`
* 对于位置 i，只能注意到 j ≤ i 的 token
* 用在 `self_attention` 中的 mask 参数

**示例（seq\_len=4）**：

| i\j | 0 | 1 | 2 | 3 |
| --- | - | - | - | - |
| 0   | 1 | 0 | 0 | 0 |
| 1   | 1 | 1 | 0 | 0 |
| 2   | 1 | 1 | 1 | 0 |
| 3   | 1 | 1 | 1 | 1 |

* 1 表示允许注意
* 0 表示屏蔽未来

**作用**：确保生成时 **不泄露未来信息**。

---

### 2️⃣ `src_mask`（Source Mask / Encoder-Decoder 注意力掩码）

**用途**：

* 用于 **Cross-Attention**（Decoder 查询 Encoder 输出时）
* 目的是 **屏蔽 Encoder 输入中的 padding token**，避免模型把无意义的 padding 当作有用信息

**形式**：

* 形状 `(N, 1, 1, src_seq_len)` 或 `(N, 1, tgt_seq_len, src_seq_len)`
* 对应 Encoder 输入的每个 token，1 表示可注意，0 表示屏蔽

**示例（src\_seq\_len=5, padding 在最后两个位置）**：

| token 0 | token 1 | token 2 | token 3 | token 4 |
| ------- | ------- | ------- | ------- | ------- |
| 1       | 1       | 1       | 0       | 0       |

* Decoder 在做 cross-attention 时，不会关注 padding

**作用**：避免模型学习无意义的 padding 信息，提升训练效果。

---

### 🔹 总结对比

| Mask 类型    | 使用场景                              | 屏蔽目标                  | 作用                   |
| ---------- | --------------------------------- | --------------------- | -------------------- |
| `tgt_mask` | Decoder 自注意力                      | 未来位置                  | 保证自回归生成，只依赖已生成 token |
| `src_mask` | Decoder → Encoder Cross-Attention | Encoder padding token | 避免关注无效信息，提高注意力效率     |

In [6]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, qk_dim, ff_hidden_size, dropout=0.1):
        super().__init__()
        # 1️⃣ Masked Self-Attention（Decoder 内部自注意力）
        self.self_attention = SelfAttention(embed_size, heads, qk_dim)
        self.norm1 = nn.LayerNorm(embed_size)
        self.dropout1 = nn.Dropout(dropout)

        # 2️⃣ Cross-Attention（Decoder 查询 Encoder 输出）
        self.cross_attention = SelfAttention(embed_size, heads, qk_dim)
        self.norm2 = nn.LayerNorm(embed_size)
        self.dropout2 = nn.Dropout(dropout)

        # 3️⃣ Feed-Forward Network
        self.feed_forward = FeedForward(embed_size, ff_hidden_size, dropout)
        self.norm3 = nn.LayerNorm(embed_size)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask=None, tgt_mask=None):
        """
        x:        (N, tgt_seq_len, embed_size)  Decoder 输入
        enc_out:  (N, src_seq_len, embed_size)  Encoder 输出
        src_mask: (N, 1, 1, src_seq_len)        Encoder-Decoder 注意力 mask
        tgt_mask: (N, 1, tgt_seq_len, tgt_seq_len)  Decoder 自注意力 mask
        """

        # -----------------------------
        # 1️⃣ Masked Self-Attention
        # Q/K/V 都来自 x
        # tgt_mask 用于屏蔽未来信息（下三角 mask）
        # 输出维度: (N, tgt_seq_len, embed_size)
        # -----------------------------
        _x = self.self_attention(query=x, key=None, value=None, mask=tgt_mask)
        x = self.norm1(x + self.dropout1(_x))  # 残差连接 + LayerNorm

        # -----------------------------
        # 2️⃣ Cross-Attention
        # Q 来自 Decoder 当前输入 x
        # K/V 来自 Encoder 输出 enc_out
        # src_mask 用于屏蔽 padding token
        # 输出维度: (N, tgt_seq_len, embed_size)
        # -----------------------------
        _x = self.cross_attention(query=x, key=enc_out, value=enc_out, mask=src_mask)
        x = self.norm2(x + self.dropout2(_x))  # 残差连接 + LayerNorm

        # -----------------------------
        # 3️⃣ Feed-Forward Network
        # 输出维度: (N, tgt_seq_len, embed_size)
        # -----------------------------
        _x = self.feed_forward(x)
        x = self.norm3(x + self.dropout3(_x))  # 残差连接 + LayerNorm

        return x


## Transformer Decoder

In [7]:
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_layers, heads, qk_dim,
                 ff_hidden_size, dropout=0.1, max_length=100):
        super().__init__()
        self.embed_size = embed_size

        # 1️⃣ 词嵌入层，将 token id 转换为向量
        self.word_embedding = nn.Embedding(vocab_size, embed_size)

        # 2️⃣ 位置编码，加入位置信息
        self.position_encoding = PositionalEncoding(embed_size, max_length)

        # 3️⃣ 多层 DecoderBlock
        self.layers = nn.ModuleList([
            DecoderBlock(embed_size, heads, qk_dim, ff_hidden_size, dropout)
            for _ in range(num_layers)
        ])

        # 4️⃣ 输入 dropout
        self.dropout = nn.Dropout(dropout)

        # 5️⃣ 输出投影层，将 embed_size 映射到 vocab_size
        self.fc_out = nn.Linear(embed_size, vocab_size)

    def forward(self, x, enc_out, src_mask=None, tgt_mask=None):
        """
        x: (N, tgt_seq_len) 目标序列 token id
        enc_out: (N, src_seq_len, embed_size) Encoder 输出
        src_mask: (N, 1, 1, src_seq_len) Encoder padding mask
        tgt_mask: (N, 1, tgt_seq_len, tgt_seq_len) Decoder 自注意力 mask
        """

        # 1️⃣ token embedding + positional encoding
        out = self.word_embedding(x)            # (N, tgt_seq_len, embed_size)
        out = self.position_encoding(out)       # 加入位置信息
        out = self.dropout(out)

        # 2️⃣ 多层 DecoderBlock 堆叠
        for layer in self.layers:
            out = layer(out, enc_out, src_mask, tgt_mask)
            # out 维度保持 (N, tgt_seq_len, embed_size)

        # 3️⃣ 输出投影到 vocab
        out = self.fc_out(out)                  # (N, tgt_seq_len, vocab_size)

        return out


有区别的，虽然在形式上都是 `(N, seq_len)` 的 token id，但 **含义和作用不同**：

---

### 1️⃣ Encoder 的输入 `x`

* **类型**：源序列（source sequence）的 token id
* **作用**：编码源序列信息，生成上下文向量 `enc_out`
* **输入示例**：一段英文句子 `[The, cat, sits, .]` → `[101, 205, 330, 102]`
* **特点**：

  * 所有 token 都可用
  * Encoder 通常不屏蔽未来 token
  * 可加 `src_mask` 屏蔽 padding

---

### 2️⃣ Decoder 的输入 `x`

* **类型**：目标序列（target sequence）的 token id
* **作用**：

  * 用于 **训练阶段**：做自回归预测，生成下一个 token
  * 用于 **推理阶段**：从 `<SOS>` 开始，逐步生成 token
* **输入示例**：翻译任务中目标句子 `[Le, chat, est, assis]` → `[101, 400, 500, 102]`
* **特点**：

  * 需要 **Masked Self-Attention** (`tgt_mask`) 避免看到未来 token
  * Cross-Attention 查询 Encoder 输出（`enc_out`）

---

#### 🔹 总结

| 模块      | 输入 `x` 类型                        | 作用                               |
| ------- | -------------------------------- | -------------------------------- |
| Encoder | 源序列 token id `(N, src_seq_len)`  | 编码源序列上下文，生成 `enc_out`            |
| Decoder | 目标序列 token id `(N, tgt_seq_len)` | 自回归预测下一 token，结合 Encoder 上下文生成输出 |

---

如果你愿意，我可以画一张 **Encoder 和 Decoder 输入输出示意图**，直观显示 `x`、`enc_out`、`tgt_mask`、`src_mask` 之间的关系，这样理解会更清楚。

你希望我画吗？


## Transformer

src ──► Encoder ──► enc_out  
tgt ──► Decoder(x = tgt, enc_out, src_mask, tgt_mask) ──► out

In [8]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, embed_size=256, num_layers=3,
                 heads=8, qk_dim=64, ff_hidden_size=512, dropout=0.1, max_length=100):
        """
        src_vocab_size: 源语言词表大小
        tgt_vocab_size: 目标语言词表大小
        embed_size: Embedding/模型维度 (d_model)
        num_layers: Encoder/Decoder 层数
        heads: 注意力头数
        qk_dim: Q/K 投影维度
        ff_hidden_size: 前馈网络隐藏层大小
        dropout: Dropout 概率
        max_length: 序列最大长度
        """
        super().__init__()

        # 1️⃣ Encoder
        self.encoder = TransformerEncoder(
            vocab_size=src_vocab_size,
            embed_size=embed_size,
            num_layers=num_layers,
            heads=heads,
            qk_dim=qk_dim,
            ff_hidden_size=ff_hidden_size,
            dropout=dropout,
            max_length=max_length
        )

        # 2️⃣ Decoder
        self.decoder = TransformerDecoder(
            vocab_size=tgt_vocab_size,
            embed_size=embed_size,
            num_layers=num_layers,
            heads=heads,
            qk_dim=qk_dim,
            ff_hidden_size=ff_hidden_size,
            dropout=dropout,
            max_length=max_length
        )

    # -----------------------------
    # 构造源序列 mask
    # -----------------------------
    def make_src_mask(self, src):
        """
        src: (N, src_seq_len)
        src_mask: (N, 1, 1, src_seq_len)
        mask 为 1 的位置表示有效 token，为 0 的位置表示 padding
        """
        return (src != 0).unsqueeze(1).unsqueeze(2)

    # -----------------------------
    # 构造目标序列 mask
    # -----------------------------
    def make_tgt_mask(self, tgt):
        """
        tgt: (N, tgt_seq_len)
        tgt_mask: (N, 1, tgt_seq_len, tgt_seq_len)
        mask 为下三角矩阵，防止 Decoder 看到未来 token
        """
        N, tgt_len = tgt.shape
        # 下三角矩阵
        tgt_mask = torch.tril(torch.ones((tgt_len, tgt_len))).expand(N, 1, tgt_len, tgt_len)
        return tgt_mask.to(tgt.device)

    # -----------------------------
    # 前向传播
    # -----------------------------
    def forward(self, src, tgt):
        """
        src: (N, src_seq_len) 源序列 token id
        tgt: (N, tgt_seq_len) 目标序列 token id
        """
        # 1️⃣ 构造 mask
        src_mask = self.make_src_mask(src)
        tgt_mask = self.make_tgt_mask(tgt)

        # 2️⃣ Encoder 编码源序列
        enc_out = self.encoder(src, mask=src_mask)  # (N, src_seq_len, embed_size)

        # 3️⃣ Decoder 解码目标序列
        out = self.decoder(tgt, enc_out, src_mask=src_mask, tgt_mask=tgt_mask)
        # 输出维度: (N, tgt_seq_len, tgt_vocab_size)

        return out


In [9]:
# 示例：构造输入并运行 Transformer

import torch

# 假设词表大小为 1000，源序列和目标序列长度均为 6
src_vocab_size = 1000
tgt_vocab_size = 1000
src_seq_len = 6
tgt_seq_len = 6
batch_size = 2

# 构造随机 token id（模拟输入）
src = torch.randint(1, src_vocab_size, (batch_size, src_seq_len))
tgt = torch.randint(1, tgt_vocab_size, (batch_size, tgt_seq_len))

# 实例化模型
model = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    embed_size=32,        # 可以调小以便快速实验
    num_layers=2,
    heads=4,
    qk_dim=32,
    ff_hidden_size=64,
    dropout=0.1,
    max_length=20
)

# 前向推理
out = model(src, tgt)

print("src shape:", src.shape)
print("tgt shape:", tgt.shape)
print("模型输出 shape:", out.shape)  # (batch_size, tgt_seq_len, tgt_vocab_size)
print("输出示例（部分）：", out[0, 0, :5].detach().numpy())

src shape: torch.Size([2, 6])
tgt shape: torch.Size([2, 6])
模型输出 shape: torch.Size([2, 6, 1000])
输出示例（部分）： [-0.2812156   0.4754691   0.96160126  0.35761005  0.6679771 ]
